In [11]:
import urllib
from bs4 import BeautifulSoup

In [69]:
class Scraper:
    def __init__(self):
        pass
        
class BailiiScraper(Scraper):
    def parse(self):
        http = urllib3.PoolManager()
        page = http.request('GET', 'http://www.bailii.org/ew/cases/EWHC/Comm/1995/1.html')
        soup = BeautifulSoup(page.data)
        return soup.body.find('p', text='B e f o r e :').find_all_next(string=False)

In [70]:
bailii = BailiiScraper()
print(bailii.parse())

[<p><b><panel>LORD JUSTICE STUART-SMITH<br/>LORD JUSTICE HOBHOUSE<br/>LORD JUSTICE MILLETT</panel></b><br/>
____________________</p>, <b><panel>LORD JUSTICE STUART-SMITH<br/>LORD JUSTICE HOBHOUSE<br/>LORD JUSTICE MILLETT</panel></b>, <panel>LORD JUSTICE STUART-SMITH<br/>LORD JUSTICE HOBHOUSE<br/>LORD JUSTICE MILLETT</panel>, <br/>, <br/>, <br/>, <parties><table width="100%">
<tr>
<td width="19%"> </td>
<td align="CENTER"><b><font size="4"> GRUPO TORRAS S.A. TORRAS HOSTENCH LONDON LIMITED<br/></font></b></td>
<td valign="BOTTOM" width="19%"><b>(Respondents)<br/></b></td>
</tr>
<tr>
<td width="19%"> </td>
<td align="CENTER"><b><font size="4">- and -<br/></font></b></td>
<td valign="BOTTOM" width="19%"><b><br/></b></td>
</tr>
<tr>
<td width="19%"> </td>
<td align="CENTER"><b><font size="4">SHEIK FARAD MOHAMMED AL SABAH<br/></font></b></td>
<td valign="BOTTOM" width="19%"><b>(1st Deft.)<br/></b></td>
</tr>
<tr>
<td width="19%"> </td>
<td align="CENTER"><b><font size="4">RICHARD ROBINSON<br

/home/user/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/user/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
